# Iris 꽃 분류 딥러닝 모델

## 🌸 프로젝트 개요
이 노트북에서는 Scikit-learn의 Iris 데이터셋을 사용하여 꽃의 품종(Setosa, Versicolor, Virginica)을 분류하는 딥러닝 모델을 구현합니다.

## 📚 학습 목표
- Scikit-learn 데이터셋을 딥러닝에 활용하는 방법
- 원-핫 인코딩(One-Hot Encoding)의 필요성과 적용
- StandardScaler를 사용한 데이터 정규화
- 작은 데이터셋에 대한 딥러닝 모델 설계

## 📁 데이터셋 정보
- **데이터셋**: Iris 꽃 데이터셋 (150개 샘플)
- **특성**: 4개 (꽃받침 길이/너비, 꽃잎 길이/너비)
- **클래스**: 3개 (Setosa, Versicolor, Virginica)
- **분할**: 훈련 70% / 테스트 30%

## 1. 라이브러리 임포트 및 시드 설정

딥러닝 모델 구현과 데이터 전처리에 필요한 라이브러리를 임포트하고 재현 가능한 결과를 위해 랜덤 시드를 설정합니다.

In [1]:
# 데이터 처리 및 분할 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 딥러닝 라이브러리
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# 재현 가능한 결과를 위한 랜덤 시드 설정
tf.random.set_seed(1)

print("TensorFlow 버전:", tf.__version__)
print("✅ 라이브러리 임포트 완료!")

TensorFlow 버전: 2.19.0
✅ 라이브러리 임포트 완료!


## 2. Iris 데이터셋 로드 및 탐색

Scikit-learn의 Iris 데이터셋을 로드하고 데이터의 구조와 특성을 확인합니다.

In [2]:
# Iris 데이터셋 로드
iris = load_iris()

# 특성(X)과 타겟(y) 분리
X = iris['data']      # 꽃의 특성 데이터 (4개 특성)
y = iris['target']    # 꽃의 품종 라벨 (3개 클래스)

# 데이터셋 기본 정보 확인
print("📊 Iris 데이터셋 기본 정보:")
print(f"특성 데이터 형태: {X.shape}")
print(f"타겟 데이터 형태: {y.shape}")
print(f"특성 이름: {iris.feature_names}")
print(f"클래스 이름: {iris.target_names}")

print(f"\n📈 클래스별 분포:")
import numpy as np
unique, counts = np.unique(y, return_counts=True)
for i, (cls, count) in enumerate(zip(iris.target_names, counts)):
    print(f"   {i}: {cls} - {count}개")

print(f"\n✅ 데이터셋 로드 완료! (총 {len(X)}개 샘플)")

📊 Iris 데이터셋 기본 정보:
특성 데이터 형태: (150, 4)
타겟 데이터 형태: (150,)
특성 이름: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
클래스 이름: ['setosa' 'versicolor' 'virginica']

📈 클래스별 분포:
   0: setosa - 50개
   1: versicolor - 50개
   2: virginica - 50개

✅ 데이터셋 로드 완료! (총 150개 샘플)


## 3. 데이터 분할 (훈련/테스트)

모델의 일반화 성능을 평가하기 위해 데이터를 훈련셋과 테스트셋으로 분할합니다.

In [3]:
# 데이터를 훈련셋과 테스트셋으로 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    random_state=1,    # 재현 가능한 결과를 위한 시드
    test_size=0.3      # 테스트셋 비율 30%
)

# 분할 결과 확인
print("📊 데이터 분할 결과:")
print(f"훈련 특성 데이터: {X_train.shape}")
print(f"훈련 타겟 데이터: {y_train.shape}")
print(f"테스트 특성 데이터: {X_test.shape}")
print(f"테스트 타겟 데이터: {y_test.shape}")

print(f"\n📈 분할 비율:")
print(f"훈련셋: {len(X_train)}개 ({len(X_train)/len(X)*100:.1f}%)")
print(f"테스트셋: {len(X_test)}개 ({len(X_test)/len(X)*100:.1f}%)")

print("\n✅ 데이터 분할 완료!")

📊 데이터 분할 결과:
훈련 특성 데이터: (105, 4)
훈련 타겟 데이터: (105,)
테스트 특성 데이터: (45, 4)
테스트 타겟 데이터: (45,)

📈 분할 비율:
훈련셋: 105개 (70.0%)
테스트셋: 45개 (30.0%)

✅ 데이터 분할 완료!


## 4. 특성 데이터 정규화 (StandardScaler)

딥러닝 모델의 학습 효율성을 높이기 위해 특성 데이터를 표준화합니다.

### 정규화가 필요한 이유:
- 각 특성의 스케일이 다를 때 학습이 불안정해질 수 있음
- 경사하강법의 수렴 속도를 향상시킴
- 모든 특성을 동등하게 처리하여 편향 방지

In [4]:
# 정규화 전 데이터 범위 확인
print("정규화 전 데이터 범위:")
print(f"X_train 최솟값: {X_train.min(axis=0)}")
print(f"X_train 최댓값: {X_train.max(axis=0)}")

# StandardScaler를 사용한 정규화
scaler = StandardScaler()

# 훈련 데이터로 스케일러를 학습하고 변환
X_train_scaled = scaler.fit_transform(X_train)

# 테스트 데이터는 훈련 데이터의 스케일로 변환 (fit 없이 transform만)
# ⚠️ 주의: 원본 코드에서는 fit_transform을 사용했지만, 
# 올바른 방법은 transform만 사용하는 것입니다.
X_test_scaled = scaler.transform(X_test)

# 정규화 후 데이터 범위 확인
print(f"\n정규화 후 데이터 범위:")
print(f"X_train_scaled 평균: {X_train_scaled.mean(axis=0).round(3)}")
print(f"X_train_scaled 표준편차: {X_train_scaled.std(axis=0).round(3)}")

print(f"\n📊 데이터 형태 확인:")
print(f"훈련 데이터: {X_train_scaled.shape}")
print(f"테스트 데이터: {X_test_scaled.shape}")

print("\n✅ 특성 데이터 정규화 완료!")
print("📝 StandardScaler: 평균=0, 표준편차=1로 정규화")

정규화 전 데이터 범위:
X_train 최솟값: [4.3 2.  1.  0.1]
X_train 최댓값: [7.7 4.4 6.9 2.5]

정규화 후 데이터 범위:
X_train_scaled 평균: [-0. -0.  0. -0.]
X_train_scaled 표준편차: [1. 1. 1. 1.]

📊 데이터 형태 확인:
훈련 데이터: (105, 4)
테스트 데이터: (45, 4)

✅ 특성 데이터 정규화 완료!
📝 StandardScaler: 평균=0, 표준편차=1로 정규화


## 5. 타겟 데이터 원-핫 인코딩

딥러닝에서 다중분류를 위해 정수 라벨을 원-핫 인코딩 형태로 변환합니다.

### 원-핫 인코딩이란?
- **정수 라벨**: [0, 1, 2]
- **원-핫 인코딩**: [[1,0,0], [0,1,0], [0,0,1]]
- categorical_crossentropy 손실함수 사용 시 필수

In [5]:
# 원-핫 인코딩 전 라벨 확인
print("원-핫 인코딩 전:")
print(f"y_train 형태: {y_train.shape}")
print(f"y_train 샘플: {y_train[:10]}")
print(f"고유값: {np.unique(y_train)}")

# 타겟 라벨을 원-핫 인코딩으로 변환
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# 원-핫 인코딩 후 라벨 확인
print(f"\n원-핫 인코딩 후:")
print(f"y_train_encoded 형태: {y_train_encoded.shape}")
print(f"y_train_encoded 샘플:")
print(y_train_encoded[:5])

print(f"\n📊 변환 결과:")
print(f"훈련 라벨: {y_train_encoded.shape}")
print(f"테스트 라벨: {y_test_encoded.shape}")

# 변환 확인 예시
print(f"\n📝 변환 예시:")
for i in range(3):
    original = np.where(y_train == i)[0][0]  # 각 클래스의 첫 번째 샘플 인덱스
    print(f"원본 라벨 {i} → 원-핫: {y_train_encoded[original]}")

print("\n✅ 원-핫 인코딩 완료!")

원-핫 인코딩 전:
y_train 형태: (105,)
y_train 샘플: [2 0 0 0 1 0 0 2 2 2]
고유값: [0 1 2]

원-핫 인코딩 후:
y_train_encoded 형태: (105, 3)
y_train_encoded 샘플:
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]

📊 변환 결과:
훈련 라벨: (105, 3)
테스트 라벨: (45, 3)

📝 변환 예시:
원본 라벨 0 → 원-핫: [1. 0. 0.]
원본 라벨 1 → 원-핫: [0. 1. 0.]
원본 라벨 2 → 원-핫: [0. 0. 1.]

✅ 원-핫 인코딩 완료!


## 6. 딥러닝 모델 구조 설계

Iris 데이터셋 분류를 위한 다층 퍼셉트론 모델을 설계합니다.

### 모델 구조:
- **입력층**: 4개 뉴런 (4개 특성)
- **은닉층 1**: 64개 뉴런 + ReLU 활성화 함수
- **은닉층 2**: 64개 뉴런 + ReLU 활성화 함수  
- **은닉층 3**: 128개 뉴런 + ReLU 활성화 함수
- **출력층**: 3개 뉴런 + Softmax 활성화 함수 (3개 클래스)

In [6]:
# Sequential 모델을 사용하여 신경망 구조 설계
network = keras.models.Sequential([
    # 은닉층 1: 64개 뉴런, ReLU 활성화 함수
    keras.layers.Dense(64, activation='relu'),
    
    # 은닉층 2: 64개 뉴런, ReLU 활성화 함수 (추가 은닉층)
    keras.layers.Dense(64, activation='relu'),
    
    # 은닉층 3: 128개 뉴런, ReLU 활성화 함수 (더 많은 뉴런으로 복잡성 증가)
    keras.layers.Dense(128, activation='relu'),
    
    # 출력층: 3개 뉴런 (3개 클래스), Softmax 활성화 함수
    keras.layers.Dense(3, activation='softmax')
])

# 모델 요약 정보 출력
print("🏗️ 모델 구조:")
network.summary()

print("\n✅ 신경망 모델 구조 설계 완료!")
print("\n📝 모델 설계 포인트:")
print("- 작은 데이터셋이므로 과적합을 방지하기 위해 적당한 크기의 네트워크 사용")
print("- ReLU 활성화 함수로 학습 효율성 증대")
print("- Softmax 출력층으로 다중분류 확률 출력")

🏗️ 모델 구조:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


✅ 신경망 모델 구조 설계 완료!

📝 모델 설계 포인트:
- 작은 데이터셋이므로 과적합을 방지하기 위해 적당한 크기의 네트워크 사용
- ReLU 활성화 함수로 학습 효율성 증대
- Softmax 출력층으로 다중분류 확률 출력


## 7. 모델 컴파일 설정

모델을 학습할 수 있도록 옵티마이저, 손실함수, 평가지표를 설정합니다.

### 설정 요소:
- **옵티마이저**: RMSprop (가중치 업데이트 방법)
- **손실함수**: categorical_crossentropy (원-핫 인코딩된 다중분류용)
- **평가지표**: accuracy (정확도)

In [7]:
# 모델 컴파일 설정
network.compile(
    optimizer='rmsprop',                  # 옵티마이저: RMSprop
    loss='categorical_crossentropy',      # 손실함수: 원-핫 인코딩된 다중분류용
    metrics=['accuracy']                  # 평가지표: 정확도
)

print("✅ 모델 컴파일 완료!")
print("\n📝 손실함수 비교:")
print("- categorical_crossentropy: 라벨이 원-핫 인코딩 형태 ([1,0,0], [0,1,0], ...)")
print("- sparse_categorical_crossentropy: 라벨이 정수 형태 (0, 1, 2, ...)")
print("  → 현재는 라벨을 원-핫 인코딩했으므로 categorical_crossentropy 사용")

print(f"\n🎯 모델 학습 준비 완료!")
print(f"입력 데이터: {X_train_scaled.shape}")
print(f"출력 데이터: {y_train_encoded.shape}")

✅ 모델 컴파일 완료!

📝 손실함수 비교:
- categorical_crossentropy: 라벨이 원-핫 인코딩 형태 ([1,0,0], [0,1,0], ...)
- sparse_categorical_crossentropy: 라벨이 정수 형태 (0, 1, 2, ...)
  → 현재는 라벨을 원-핫 인코딩했으므로 categorical_crossentropy 사용

🎯 모델 학습 준비 완료!
입력 데이터: (105, 4)
출력 데이터: (105, 3)


## 8. 모델 학습 실행

fit() 함수를 사용하여 모델을 학습시킵니다.

### 학습 매개변수:
- **epochs**: 30 (전체 데이터셋을 30번 반복 학습)
- **batch_size**: 100 (한 번에 100개 샘플씩 처리)

In [8]:
# 모델 학습 실행
print("🚀 모델 학습을 시작합니다...")
print("⏰ Iris 데이터셋은 작으므로 빠르게 학습됩니다.")

history = network.fit(
    X_train_scaled,      # X: 정규화된 훈련 특성 데이터
    y_train_encoded,     # y: 원-핫 인코딩된 훈련 라벨
    epochs=30,           # 학습 회수 (30 에포크)
    batch_size=100,      # 배치 크기 (전체 데이터보다 큰 값이므로 실제로는 전체 배치)
    verbose=1            # 학습 진행 상황 출력
)

print("\n✅ 모델 학습 완료!")
print("\n📝 배치 크기 참고사항:")
print(f"- 설정된 배치 크기: 100")
print(f"- 실제 훈련 데이터 크기: {len(X_train_scaled)}")
print("- 훈련 데이터가 배치 크기보다 작으므로 실제로는 전체 배치로 학습됨")

🚀 모델 학습을 시작합니다...
⏰ Iris 데이터셋은 작으므로 빠르게 학습됩니다.
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.0578 - loss: 1.1517 
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5613 - loss: 1.0150
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6063 - loss: 0.9203
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6838 - loss: 0.8377
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7806 - loss: 0.7653
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8871 - loss: 0.7038
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9419 - loss: 0.6502
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9419 - loss: 0.6035
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9225 - loss: 0.5602
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9129 - loss: 0.5205
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9225 - loss: 0.4840
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 

## 9. 모델 성능 평가

evaluate() 함수를 사용하여 훈련셋과 테스트셋에서의 모델 성능을 평가합니다.

In [9]:
# 훈련셋 성능 평가
train_loss, train_acc = network.evaluate(X_train_scaled, y_train_encoded, verbose=0)
print("📊 훈련셋 성능:")
print(f"   손실(Loss): {train_loss:.4f}")
print(f"   정확도(Accuracy): {train_acc:.4f} ({train_acc*100:.2f}%)")

# 테스트셋 성능 평가
test_loss, test_acc = network.evaluate(X_test_scaled, y_test_encoded, verbose=0)
print("\n📊 테스트셋 성능:")
print(f"   손실(Loss): {test_loss:.4f}")
print(f"   정확도(Accuracy): {test_acc:.4f} ({test_acc*100:.2f}%)")

# 성능 차이 분석
accuracy_diff = train_acc - test_acc
print(f"\n📈 성능 분석:")
print(f"   정확도 차이: {accuracy_diff:.4f} ({accuracy_diff*100:.2f}%p)")

if accuracy_diff > 0.1:  # 10% 이상 차이
    print("   ⚠️  과적합(Overfitting) 가능성이 있습니다.")
elif accuracy_diff < -0.05:  # 테스트가 훈련보다 5% 이상 좋음
    print("   🤔 테스트셋 성능이 더 좋습니다. 데이터 분할을 다시 확인해보세요.")
else:
    print("   ✅ 적절한 일반화 성능을 보입니다.")

print(f"\n🎉 Iris 꽃 분류 딥러닝 모델 구현 완료!")
print(f"📊 최종 테스트 정확도: {test_acc*100:.1f}%")

📊 훈련셋 성능:
   손실(Loss): 0.1430
   정확도(Accuracy): 0.9524 (95.24%)

📊 테스트셋 성능:
   손실(Loss): 0.2610
   정확도(Accuracy): 0.8667 (86.67%)

📈 성능 분석:
   정확도 차이: 0.0857 (8.57%p)
   ✅ 적절한 일반화 성능을 보입니다.

🎉 Iris 꽃 분류 딥러닝 모델 구현 완료!
📊 최종 테스트 정확도: 86.7%


## 📚 학습 요약 및 MNIST와의 비교

### ✅ Iris 딥러닝 구현 내용:
1. **Scikit-learn 데이터셋 로드**
2. **데이터 분할 (70% 훈련 / 30% 테스트)**
3. **StandardScaler를 이용한 특성 정규화**
4. **원-핫 인코딩을 통한 라벨 변환**
5. **다층 퍼셉트론 모델 설계**
6. **모델 학습 및 성능 평가**

### 🔄 MNIST와 Iris의 차이점:

| 구분 | MNIST | Iris |
|------|-------|------|
| **데이터 크기** | 70,000개 | 150개 |
| **입력 차원** | 784 (28×28 이미지) | 4 (수치형 특성) |
| **클래스 수** | 10개 (0~9 숫자) | 3개 (꽃 품종) |
| **전처리** | 차원변환 + 0~1 정규화 | StandardScaler 정규화 |
| **라벨 형태** | 정수 → sparse_categorical_crossentropy | 원-핫 → categorical_crossentropy |
| **학습 시간** | 상대적으로 오래 걸림 | 매우 빠름 |

### 🎯 딥러닝 vs 전통적 머신러닝:
- **작은 데이터셋 (Iris)**: 전통적 머신러닝이 더 효과적일 수 있음
- **큰 데이터셋 (MNIST)**: 딥러닝의 장점이 더 잘 드러남
- **복잡한 패턴**: 딥러닝이 우수한 성능 발휘

### 📝 핵심 학습 포인트:
1. **데이터 전처리의 중요성** (정규화, 인코딩)
2. **손실함수 선택** (sparse vs categorical)
3. **모델 크기 조절** (데이터 크기에 맞는 네트워크)
4. **과적합 주의** (작은 데이터셋에서 특히 중요)